In [1]:
import pandas as pd
import psycopg2
import datetime
import argparse

In [2]:
def run_queries(date):    
    # Establish a connection to the PostgreSQL database
    conn = psycopg2.connect(
        host="10.7.8.59",
        database="fixtransactions",
        user="scalp",
        password="QAtr@de442",
        port="5433"
    )

    # Define the SQL queries
    query1 = f"""
    CREATE TEMP TABLE outgoing AS
    SELECT 
        tag167,
        tag55,
        tag11,
        tag49,
        tag56,
        tag52,
        tag100
    FROM fixmsg
    WHERE 
        tag35 = 'D'
        AND tag167 = 'OPT'
        AND tag52 >= TIMESTAMP '{date}'
        AND tag52 < TIMESTAMP '{date}' + INTERVAL '1 DAY'
    """

    query2 = f"""
    CREATE TEMP TABLE incoming AS
    SELECT 
        tag167,
        tag55,
        tag11,
        tag49,
        tag56,
        tag52,
        tag100
    FROM fixmsg
    WHERE 
        tag35 = '8'
        AND tag39 = '0'
        AND tag52 >= TIMESTAMP '{date}'
        AND tag52 < TIMESTAMP '{date}' + INTERVAL '1 DAY'
    """
    
    query3 = f"""
    CREATE TEMP TABLE results AS
    SELECT 
        i.tag11 AS tag11,
        i.tag52 AS cancelack_52,
        o.tag100 AS tag100,
        o.tag11 AS tag11_a,
        o.tag56 AS target_compid,
        o.tag52 AS cancel_52
    FROM incoming i
    INNER JOIN outgoing o
        ON o.tag11 = i.tag11;

    """

    # Execute the queries
    with conn.cursor() as cursor:
        cursor.execute(query1)
        cursor.execute(query2)
        cursor.execute(query3)

    # Load the query results into dataframes
    table2_df = pd.read_sql_query("SELECT * FROM results", conn)

    # Write dataframes to CSV files with date in the file names
    table2_df.to_csv(f"/home/elliott/development/files/options_nack_latencies_{date}.csv", index=False)
    
    # Close the database connection
    conn.close()

In [43]:
run_queries('2023-03-10')

/tmp/ipykernel_203501/2981929255.py:70: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  table2_df = pd.read_sql_query("SELECT * FROM results", conn)


In [44]:
table2_df = pd.read_csv('/home/elliott/development/files/options_nack_latencies_2023-03-10.csv')
table2_df

,tag11,cancelack_52,tag100,tag11_a,target_compid,cancel_52
0,320043802057574,2023-03-10 14:30:04.931,PCXO,320043802057574,INCAPNS,2023-03-10 14:30:04.930
1,320043802057575,2023-03-10 14:30:05.147,PCXO,320043802057575,INCAPNS,2023-03-10 14:30:05.146
2,320043802057578,2023-03-10 14:30:05.647,PCXO,320043802057578,INCAPNS,2023-03-10 14:30:05.646
3,320043802057580,2023-03-10 14:30:05.795,PCXO,320043802057580,INCAPNS,2023-03-10 14:30:05.794
4,320043802057582,2023-03-10 14:30:06.064,PCXO,320043802057582,INCAPNS,2023-03-10 14:30:06.063
...,...,...,...,...,...,...
154872,380043868900551,2023-03-10 20:38:36.889,PCXO,380043868900551,INCAPNS,2023-03-10 20:38:36.888
154873,380043868900797,2023-03-10 20:38:37.571,PCXO,380043868900797,INCAPNS,2023-03-10 20:38:37.570
154874,380043868901144,2023-03-10 20:38:38.602,PCXO,380043868901144,INCAPNS,2023-03-10 20:38:38.601
154875,380043868901357,2023-03-10 20:38:39.231,PCXO,380043868901357,INCAPNS,2023-03-10 20:38:39.231


In [45]:
table2_df['cancel_52'] = pd.to_datetime(table2_df['cancel_52'])
table2_df['cancelack_52'] = pd.to_datetime(table2_df['cancelack_52'])

In [46]:
table2_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 154877 entries, 0 to 154876
Data columns (total 6 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   tag11          154877 non-null  int64         
 1   cancelack_52   154877 non-null  datetime64[ns]
 2   tag100         154877 non-null  object        
 3   tag11_a        154877 non-null  int64         
 4   target_compid  154877 non-null  object        
 5   cancel_52      154877 non-null  datetime64[ns]
dtypes: datetime64[ns](2), int64(2), object(2)
memory usage: 7.1+ MB


In [47]:
table2_df["time_diff"] = (table2_df["cancelack_52"] - table2_df["cancel_52"]).apply(lambda x: x.total_seconds())
table2_df = table2_df.sort_values(by='time_diff', ascending=False)

In [48]:
table2_df

,tag11,cancelack_52,tag100,tag11_a,target_compid,cancel_52,time_diff
862,320043802059508,2023-03-10 14:31:54.500,CBOEO,320043802059508,INCAPNS,2023-03-10 14:31:52.959,1.541
970,320043802059756,2023-03-10 14:32:10.323,CBOEO,320043802059756,INCAPNS,2023-03-10 14:32:08.906,1.417
270,320043802058228,2023-03-10 14:30:51.570,CBOEO,320043802058228,INCAPNS,2023-03-10 14:30:50.365,1.205
667,320043802059085,2023-03-10 14:31:31.892,CBOEO,320043802059085,INCAPNS,2023-03-10 14:31:30.728,1.164
149,320043802057922,2023-03-10 14:30:32.480,CBOEO,320043802057922,INCAPNS,2023-03-10 14:30:31.377,1.103
...,...,...,...,...,...,...,...
83350,320043802257514,2023-03-10 17:43:16.726,PCXO,320043802257514,INCAPNS,2023-03-10 17:43:16.726,0.000
83349,320043802257513,2023-03-10 17:43:16.647,XBXO,320043802257513,VOLANT,2023-03-10 17:43:16.647,0.000
83348,320043802257510,2023-03-10 17:43:16.390,PCXO,320043802257510,INCAPNS,2023-03-10 17:43:16.390,0.000
77438,320043802244341,2023-03-10 17:25:21.654,XBXO,320043802244341,VOLANT,2023-03-10 17:25:21.654,0.000


In [49]:
table2_df_filtered = table2_df[table2_df['time_diff'] != 0]
# calculate the threshold value
threshold = table2_df_filtered['time_diff'].quantile(0.90)

# filter the dataframe to include only values above the threshold
top_2_percent = table2_df_filtered[table2_df_filtered['time_diff'] >= threshold]

top_2_percent

,tag11,cancelack_52,tag100,tag11_a,target_compid,cancel_52,time_diff
862,320043802059508,2023-03-10 14:31:54.500,CBOEO,320043802059508,INCAPNS,2023-03-10 14:31:52.959,1.541
970,320043802059756,2023-03-10 14:32:10.323,CBOEO,320043802059756,INCAPNS,2023-03-10 14:32:08.906,1.417
270,320043802058228,2023-03-10 14:30:51.570,CBOEO,320043802058228,INCAPNS,2023-03-10 14:30:50.365,1.205
667,320043802059085,2023-03-10 14:31:31.892,CBOEO,320043802059085,INCAPNS,2023-03-10 14:31:30.728,1.164
149,320043802057922,2023-03-10 14:30:32.480,CBOEO,320043802057922,INCAPNS,2023-03-10 14:30:31.377,1.103
...,...,...,...,...,...,...,...
30738,320043802130013,2023-03-10 15:23:28.117,XBXO,320043802130013,VOLANT,2023-03-10 15:23:28.116,0.001
44084,320043802167474,2023-03-10 15:51:10.293,ARCO,320043802167474,VOLANT,2023-03-10 15:51:10.292,0.001
10575,320043802081706,2023-03-10 14:47:58.598,PCXO,320043802081706,INCAPNS,2023-03-10 14:47:58.597,0.001
43068,320043802165211,2023-03-10 15:49:04.905,PCXO,320043802165211,INCAPNS,2023-03-10 15:49:04.904,0.001


In [50]:
# group the dataframe by 'tag100' column and count the occurrences per exchange
counts = top_2_percent.groupby('tag100')['tag100'].value_counts()
single_df = counts.reset_index(level=1, drop=True)
single_df = single_df.to_frame('counts')
single_df = single_df.reset_index()
result_df = top_2_percent.merge(single_df, on='tag100', how='left')
result_df


,tag11,cancelack_52,tag100,tag11_a,target_compid,cancel_52,time_diff,counts
0,320043802059508,2023-03-10 14:31:54.500,CBOEO,320043802059508,INCAPNS,2023-03-10 14:31:52.959,1.541,134
1,320043802059756,2023-03-10 14:32:10.323,CBOEO,320043802059756,INCAPNS,2023-03-10 14:32:08.906,1.417,134
2,320043802058228,2023-03-10 14:30:51.570,CBOEO,320043802058228,INCAPNS,2023-03-10 14:30:50.365,1.205,134
3,320043802059085,2023-03-10 14:31:31.892,CBOEO,320043802059085,INCAPNS,2023-03-10 14:31:30.728,1.164,134
4,320043802057922,2023-03-10 14:30:32.480,CBOEO,320043802057922,INCAPNS,2023-03-10 14:30:31.377,1.103,134
...,...,...,...,...,...,...,...,...
57747,320043802130013,2023-03-10 15:23:28.117,XBXO,320043802130013,VOLANT,2023-03-10 15:23:28.116,0.001,2973
57748,320043802167474,2023-03-10 15:51:10.293,ARCO,320043802167474,VOLANT,2023-03-10 15:51:10.292,0.001,15710
57749,320043802081706,2023-03-10 14:47:58.598,PCXO,320043802081706,INCAPNS,2023-03-10 14:47:58.597,0.001,35390
57750,320043802165211,2023-03-10 15:49:04.905,PCXO,320043802165211,INCAPNS,2023-03-10 15:49:04.904,0.001,35390


In [51]:
def csv(date):
    result_df.to_csv(f"/home/elliott/development/files/options_cancelack_2%highest_latencies_{date}.csv", index=False, )

csv('2023-03-10')

In [52]:
# Set display options
pd.set_option('display.float_format', '{:.3f}'.format)
pd.options.display.precision = 3

result_df = table2_df.drop(['cancel_52', 'cancelack_52', 'tag41', 'counts'], axis=1, errors='ignore')
result_df['tag11'] = result_df['tag11'].astype(str)

#percentile list
perc = [.25, .50, .80, .95, .99]
  
# list of dtypes to include
include = ['object', 'float', 'int']


# calling describe method
desc = table2_df.describe(percentiles=perc, include=include)

grouped = result_df.groupby('tag100')['time_diff'].mean()

test = (table2_df.groupby('tag100')['time_diff'].mean()).describe(percentiles=perc, include=include)

#print("Percentiles:")
#print(desc)

#print("\nGrouped by tag100 (average time_diff):")
#print(grouped)

print(desc)
print(table2_df)
print(test)


                     tag11  tag100             tag11_a target_compid  \
count           154877.000  154877          154877.000        154877   
unique                 NaN      40                 NaN             3   
top                    NaN    ARCO                 NaN        VOLANT   
freq                   NaN   77724                 NaN         91168   
mean   320089515982052.812     NaN 320089515982052.812           NaN   
std      1637877860676.820     NaN   1637877860676.820           NaN   
min    320043802057574.000     NaN 320043802057574.000           NaN   
25%    320043802154466.000     NaN 320043802154466.000           NaN   
50%    320043802244341.000     NaN 320043802244341.000           NaN   
80%    320043802354314.625     NaN 320043802354314.625           NaN   
95%    320043802410079.375     NaN 320043802410079.375           NaN   
99%    320043802425283.250     NaN 320043802425283.250           NaN   
max    380043868901508.000     NaN 380043868901508.000          